# Goals, Actions, Information, Languages

* goals
    * persona
    * rules
    * process
* actions
    * available actions & tools
* information
    * task
    * history
    * inputs
    * environment state
* language

* G, A, L -> system messages, I -> user messages

## Tools

* tools & actions available to LLM, to interact with the environment
* tools (open) vs action (explicit, finite number of options)
* both need to be properly named & described/specified, to be understand and correctly leveraged by LLM
    * are tools optional or to be used in order, etc?
* feedback from tools/environment critical for further actions